In [197]:
np.random.seed(123)

#weights matrix
A=np.array([[0,1,0,0,0,0,1,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
   [0,0,0,1,1,1,0,0,0,1,0,1,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
   [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,1,1,1,0,0,0],
   [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,1,0,0,0,0,0,1,0]])

#data simulation
num_objects=100
object_dim=36
 
sigma_x_orig=0.5

I=sigma_x_orig*np.eye(object_dim)
Z_orig=np.zeros((num_objects,4))

X=np.zeros((num_objects,object_dim))

for i in range(num_objects):
    Z_orig[i,:]=(np.random.uniform(0,1,4)>0.5)
    while sum(Z_orig[i,:])==0:
        Z_orig[i,:]=(np.random.uniform(0,1,4)>0.5)
    X[i,:]=np.dot(np.random.normal(0,1,object_dim),I)+np.dot(Z_orig[i,:],A)

In [ ]:
#initial alpha, K_p, sigma_x, sigma_a
np.random.seed(123)

mcmc = 8
N = X.shape[0]
chain_alpha = np.zeros(mcmc)
chain_sigma_a = np.zeros(mcmc)
chain_sigma_x = np.zeros(mcmc)
chain_K_plus = np.zeros(mcmc)

#initial matrix Z
Z = np.array(np.random.choice(2,N,p = [0.5,0.5])).reshape(N,1)


chain_alpha[0] = alpha = 1/2
chain_sigma_a[0] = sigma_a = 0.5
chain_sigma_x[0] = sigma_x = 0.7
chain_K_plus[0] = K = 1

Hn = 0
for i in range(1,mcmc):
    #gibbs
    Hn = Hn + 1/i
    alpha = np.random.gamma(1/2+K,1/(0.5+Hn))
    for im in range(0,N): #loop over each image
        #sample K---num of new features
        new_k = new_K(alpha,N,Z,sigma_x,sigma_a,im)
        new_zi = np.zeros((N,new_k[0]))
        new_zi[im,:] = np.ones((1,new_k[0]))
        Z = np.hstack([Z,new_zi.reshape(N,new_k[0])])
        K = K + new_k[0]# attach new features to Z matrix
        
         #sample new Z_i
        for k in range(0,Z.shape[1]):
            zk_sum = np.sum(Z[0:im,k])+np.sum(Z[im+1:N,k])+1
            if zk_sum == 0:
                lz = 0
            else:
                if zk_sum == N:
                    lz0 = 0
                else:
                    lz0 = np.log(1-zk_sum/N)
                    lz = np.log(zk_sum/N)
            Z[im,k] = 1
            p1 = full_X(Z,X,sigma_x,sigma_a)+lz
            Z[im,k] = 0
            p0 = full_X(Z,X,sigma_x,sigma_a)+lz0
            if(p1 > p0):
                Z[im,k] = 1
            else:
                Z[im,k] = 0
            
            
        #sample a new sigma_x and sigma_a with MH,invgamma(2,2) prior/invgamma(1,1) proposal
#         for mh in range(0,5):
#             '''propose new sigma_x'''
#             sig_x_str = 1/np.random.gamma(1,1)#propose a new sigma_x from invgamma(1,1)
#             pos_str = full_X(Z,X,sig_x_str,sigma_a)-3*np.log(sig_x_str)-1/(2*sig_x_str)
#             pos = full_X(Z,X,sigma_x,sigma_a)-3*np.log(sigma_x)-1/(2*sigma_x)
#             if((pos_str-pos) > 0):
#                 sigma_x = sig_x_str
#             else:
#                 move = np.random.rand(1)
#                 if(np.log(move) < (pos_str-pos)):
#                     sigma_x = sig_x_str
#             '''propose new sigma_a'''
#             sig_a_str = 1/np.random.gamma(1,1)
#             pos_str = full_X(Z,X,sigma_x,sig_a_str)-3*np.log(sig_a_str)-1/(2*sig_a_str)
#             pos = full_X(Z,X,sigma_x,sigma_a)-3*np.log(sigma_a)-1/(2*sigma_a)
#             if((pos_str-pos) > 0):
#                 sigma_a = sig_a_str
#             else:
#                 move = np.random.rand(1)
#                 if(np.log(move) < (pos_str-pos)):
#                     sigma_a = sig_a_str
    
    #remove feature with only object
    
    #store chain values                
    chain_alpha[i] = alpha
    chain_sigma_a[i] = sigma_a
    chain_sigma_x[i] = sigma_x
    chain_K_plus[i] = K
    

In [146]:
#probability calculation
# X -- N*D, Z -- N*K
import numpy as np
import numpy.linalg
import math

def full_X(Z,X,s_x,s_a):
    D = X.shape[1]
    N = Z.shape[0]
    K = Z.shape[1]
    """The constant part"""
    zz = Z.T@Z+np.diag([(s_x**2)/(s_a**2)]*K) #zz -- K*K
    determ = np.linalg.det(zz)
    log_const = 0.5*N*D*np.log(2*np.pi)+(N-K)*D*np.log(sigma_x)+K*D*np.log(sigma_a)+0.5*D*np.log(determ)
    log_const = -log_const
    #const = np.exp(-log_const)
    """The exponential part"""
    ii =  np.eye(N)-Z @ np.linalg.inv(zz) @ Z.T
    tr =  np.trace(X.T @ ii @X)
    expon = -tr/(2*s_x**2)
    return(log_const+expon)
    #expon = np.exp(-tr/(2*s_x**2))
    #return(const*expon)

In [204]:
LH = full_X(Z,X,sigma_x,sigma_a)
l = alpha/N
new_zi = np.zeros((N,1))
new_zi[im,:] = np.ones((1,1))
new_Z = np.hstack([Z,new_zi.reshape(N,1)])
LH = full_X(new_Z,X,sigma_x,sigma_a)
log_prior = i*np.log(l)-l-0
a =new_K(alpha,N,Z,sigma_x,sigma_a,0)
a,a[0]

(array([6]), 6)

In [202]:
#sample the number of new dishes
def new_K(alpha,N,Z,s_x,s_a,obj):
    k_prob = np.zeros(10)
    for i in range(0,10):
        l = alpha/N
        new_zi = np.zeros((N,i))
        new_zi[obj,:] = np.ones((1,i))
        new_Z = np.hstack([Z,new_zi.reshape(N,i)])
        LH = full_X(new_Z,X,s_x, s_a)
        log_prior = i*np.log(l)-l-np.log(math.factorial(i))
        k_prob[i] = LH + log_prior#likelihood*prior = posterior
        #k_prob[i] = np.exp(k_prob[i])
        #if (k_prob[i] > 1 or k_prob[i] < 0):
        #    return(i)
    
    k_prob = k_prob/sum(k_prob)
    if (abs(sum(k_prob)-1)>0.001):
        return(sum(k_prob),'wrong k sum')
    
    new_k = np.random.choice(10,1,p = k_prob)
    return (new_k)
# s = 0
#    u = np.random.rand(1)
#    new_k = 0
# for j in range(0,10):
#        s = s+k_prob[j]
#        if u<s:
#            new_k = k_prob[j]
#            break
            
   

In [31]:
Z = np.array(np.random.choice(2,10,p = [0.5,0.5]))
Z,Z[0:3]

(array([1, 0, 0, 1, 0, 1, 0, 0, 1, 1]), array([1, 0, 0]))

In [22]:
N = 5
i=3
Z = np.eye(5)
obj = 1
new_zi = np.zeros((N,i))
new_zi[obj,:] = np.ones((1,i))
new_Z = np.hstack([Z,new_zi.reshape(N,i)])
new_Z

array([[1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 1., 1., 1.],
       [0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0.]])

In [8]:
sig_x = 1/np.random.gamma(2,0.5)
sig_x**(-2-1)/math.gamma(2) * math.exp(-1/(2*sig_x))

1.5689899151946927

In [148]:
def likelihood(X, Z, sigma_A, sigma_X, K_plus, num_objects, object_dim):
    """cythonized likelihood"""
    M1 = (np.dot(Z.T,Z) + np.dot((sigma_X**2/sigma_A**2),np.eye(K_plus)))
    log_ll=(-1)*num_objects*object_dim*0.5*np.log(2*np.pi)-1*(num_objects-K_plus)*object_dim*np.log(sigma_X)-K_plus*object_dim*np.log(sigma_A)-object_dim*(0.5)*np.log(np.linalg.det(M1))+(-1/(2*sigma_X**2))*np.trace(np.dot(np.dot(X.T,(np.eye(num_objects)-np.dot(np.dot(Z,np.linalg.inv(M1)),Z.T))),X))
    return log_ll

likelihood(X,Z,sigma_a,sigma_x,K,100,36)

-4044.8568690845223

In [200]:
Z = np.array(np.random.choice(2,N,p = [0.5,0.5])).reshape(N,1)
np.sum(Z,axis=0)

array([56])

In [210]:
np.log(1/100)

-4.605170185988091